# Локализация. Упражнения

В этих упражнениях вам предлагается реализовать простые алгоритмы локализации роботов. На лекции, помимо фильтра частиц, были упомянуты методы триангуляции и альфа-бета фильтр. Давайте их реализуем: обе реализации должны быть сделаны в виде класса, у которого должен быть метод update. Метод update принимает на вход измеренения с камеры о положении ориентиров в сосбтвенной системе координат робота.

В качестве ориентиров будем использовать координаты стоек ворот. Для удобства будем пользоваться упрощенным вариантом, при котором ворота разных цветов. В словаре храняться положения стоек в глобальной системе координат (точка (0, 0) нахождится в центре поля)

In [1]:
landmarks = {
    "blue_posts": [
        [ -4.5, -1.0], [ -4.5, 1.0]
    ],
    "yellow_posts":[
        [ 4.5, -1.0], [ 4.5, 1.0]
    ]
}

In [2]:
import numpy as np

Пример данных, которые могут поступать в модуль на одном шаге всей системы. Данные во втором словаре – это данные одометрии с собвственной информацией робота о перемещении за один шаг системы. Для проверки работы придумайте и запишите набор данных с такой же структурой. 

In [3]:
observations = {"yellow_posts":[[4.45, -0.99],[4.49, 1.09]], "blue_posts":[]}
shift = {'shift_x':0.0, 'shift_y':0.01, 'shift_angle':-0.4}

### 1) Реализовать локализацию триангуляцией (3 балл)
    

In [4]:
class Robot:
    def __init__(self, x=0, y =0, yaw=0):
        self.x = x
        self.y = y
        self.yaw = yaw
    def set_coord(self, x, y, yaw):
        self.x = float(x)
        self.y = float(y)
        self.yaw = float(yaw)
    def move(self, x,y,yaw):
        orientation = self.yaw + float(-yaw)
        if orientation < 0:
            orientation += math.pi *2
        self.yaw =  orientation % (math.pi *2)
        self.x += x*math.cos(self.yaw)
        self.y += y*math.sin(self.yaw)

In [5]:
class localization_triangulation:
    def __init__(self, x_start, y_start):
        self.x = x_start
        self.y = y_start
    
    def update(self, distances):
        def find_height_by_area(a,b,c):
            sum1=a+b+c
            return np.sqrt((sum1/2-a)*(sum1/2-b)*(sum1/2 - c)*2*sum1)/c
            
        def find_coordinates(distances,landmarks,yellow=True):
            a = distances[0]
            b = distances[1]
            l = landmarks[1][1] - landmarks[0][1]
            h=find_height_by_area(a,b,l)
            
            y = landmarks[0][0] - h if yellow else  landmarks[0][0] + h
            x= landmarks[0][1] - np.sqrt((a**2 - h**2)) if (b**2 > l**2 + a**2) else landmarks[1][1] +np.sqrt((b**2 - h**2))
            bool_b_a=  b**2 > l**2 + a**2
            bool_a_b= a**2 > l**2 + b**2
            x= x if (bool_a_b or bool_b_a) else landmarks[0][1]+np.sqrt((a**2 - h**2))
            return x,y
            
        if not distances["yellow_posts"]:
            self.x,self.y=find_coordinates(distances["blue_posts"],landmarks["blue_posts"],yellow=False)
        else:
            self.x,self.y=find_coordinates(distances["yellow_posts"],landmarks["yellow_posts"])

In [6]:
def distances_from_positions(positions):
    dst_yellow1=np.linalg.norm(positions-landmarks["yellow_posts"][0])
    dst_yellow2=np.linalg.norm(positions-landmarks["yellow_posts"][1])
    dst_blue1=np.linalg.norm(positions-landmarks["blue_posts"][0])
    dst_blue2=np.linalg.norm(positions-landmarks["blue_posts"][1])
    
    return {"yellow_posts": [dst_yellow1, dst_yellow2], "blue_posts": [dst_blue1, dst_blue2]}

In [7]:
x0=1
y0=1
positions1 = np.array([y0,x0]) #(np.array(y,x))
distances1 = distances_from_positions(positions1)
distances1["yellow_posts"] = []
Robot = localization_triangulation(0, 0)
Robot.update(distances1)
print(Robot.x==x0, Robot.y==y0)


True True


### 2) Дополнить предыдущий пункт до альфа-бета фильтра (5 балла)

In [8]:
class localization_alphabeta:
    
    def __init__(self, x_start, y_start, alpha, beta):
        self.x = x_start
        self.y = y_start
        self.local_tri =localization_triangulation(x_start, y_start)
        self.alpha = alpha
        self.beta = beta
    
    def update(self, distances, shift):
        self.local_tri.update(distances)
        self.x = self.alpha*self.local_tri.x + self.beta*(self.x+shift["x"])
        self.y = self.alpha*self.local_tri.y + self.beta*(self.y+shift["x"])

### 3) *Дополнительно* Добавьте визуализацию вида сверху  (2 балла)

Например на OpenCV или Matplotlib, например так как было в семинаре, чтобы можно было удобно посмотреть на работу модуля.
